In [21]:
from typing import Dict, List
import os
from datetime import datetime

import plotly.graph_objects as go
import plotly.express as px

def timestr_to_seconds(time_str: str) -> float:
    if 'day' in time_str:
        fmt = '%d day, %H:%M:%S.%f'
    else:
        fmt = '%H:%M:%S.%f'
    time = datetime.strptime(time_str, fmt)
    return (time - datetime(1900, 1, 1)).total_seconds()

def read_log(log_path: str) -> Dict[str, List[float | int]]:

    iters: List[int] = []
    lasted: List[float] = []
    best_costs: List[int] = []

    with open(log_path, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith('Data for iter '):
                i_iter = int(line.split('Data for iter ')[1].split(' ')[0])
                iters.append(i_iter)
            elif line.startswith('Training lasted '):
                time_str = line.split('Training lasted ')[-1][:-2]
                secs = timestr_to_seconds(time_str)
                lasted.append(secs)
            elif '_best_graph_cost' in line:
                cost = int(line.split(' ')[-1])
                best_costs.append(cost)
    
    data = {
        'iteration': iters,
        'time': lasted,
        'best cost': best_costs,
    }
    return data


def draw_figure(
    X: List[float | int], Ys: Dict[str, List[float | int]],
    title: str | None, xaxis_title: str, yaxis_title: str,
) -> go.Figure:
    fig = go.Figure()
    for y_name, y in Ys.items():
        fig.add_trace(go.Scatter(
            x=X, y=y, mode='lines', name=y_name,
        ))
    # end for
    fig.update_layout(
        title={
            'text': title,
            'x': 0.5,
        } if title else None,
        xaxis_title=xaxis_title,
        yaxis_title=yaxis_title,
        # margin=dict(l=0,r=0,b=0,t=0),
    )
    fig.update_layout(
        legend=dict(
            yanchor="top", y=0.99,
            xanchor="right", x=0.99,
        ),
    )
    # fig.update_xaxes(showgrid=False, showline=True, mirror=True, linewidth=1, linecolor='black')
    # fig.update_yaxes(showgrid=False, showline=True, mirror=True, linewidth=1, linecolor='black')
    # fig.update_layout({
    #     'plot_bgcolor': 'rgba(0, 0, 0, 0)',
    # })
    return fig


In [24]:
gf26_log = read_log('nam_8_gf2^6_mult.log')
fig = draw_figure(
    X=gf26_log['iteration'],
    Ys={
        'best cost': gf26_log['best cost'],
    },
    title=None,
    xaxis_title='Iteration',
    yaxis_title='Best cost',
)
fig.show()